<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import warnings
from collections import Counter
warnings.filterwarnings("ignore")

In [60]:
# Данные для связи с сервером
'''DBNAME = 
USER = 
PASSWORD = 
HOST = 
PORT = '''

'DBNAME = \nUSER = \nPASSWORD = \nHOST = \nPORT = '

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
# текст запроса
query_3_1 = f'''select count(*) "Количество вакансий"
                from public.vacancies
'''

In [5]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

,Количество вакансий
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
# текст запроса
query_3_2 = f'''select count(*) "Количество работодателей"
                from public.employers
'''

In [7]:
# результат запроса
df = pd.read_sql_query(query_3_2, connection)
df

,Количество работодателей
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
# текст запроса
query_3_3 = f'''select count(*) "Количество регионов"
                from public.areas
'''

In [9]:
# результат запроса
df = pd.read_sql_query(query_3_3, connection)
df

,Количество регионов
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# текст запроса
query_3_4 = f'''select count(*) "Количество сфер деятельности"
                from public.industries
'''

In [11]:
# результат запроса
df = pd.read_sql_query(query_3_4, connection)
df

,Количество сфер деятельности
0,294


***

Представляется очень небольшим количество сфер деятельности: всего 294. Есть несколько объяснений этому: 
1. Сферы деятельности обобщены и укрупнены
2. Не все работодатели указали сферу своей деятельности
3. Ряд работодателей может работать в большем количестве сфер, чем указано
4. Выборка сконцентрирована на каких-то определенных отраслях

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [12]:
# текст запроса
query_4_1 = f'''select a.name area, count(*) cnt
                from public.vacancies v
                join public.areas a on v.area_id = a.id
                group by a.id
                order by 2 desc
                limit 5
'''

In [13]:
# результат запроса
df = pd.read_sql_query(query_4_1, connection)
df

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [14]:
# текст запроса
query_4_2 = f'''select count(*)
                from public.vacancies v
                where v.salary_from is not null or v.salary_to is not null
'''

In [15]:
# результат запроса
df = pd.read_sql_query(query_4_2, connection)
df

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [16]:
# текст запроса
# Для наглядности решим задачу при помощи union. Более простое решение - указать в выдаче сразу обе агрегирующих функции
query_4_3 = f'''select round(avg(v.salary_from)) as "Средняя ЗП", 'Нижняя граница' as "Граница"
                from public.vacancies v
                union
                select round(avg(v.salary_to)), 'Верхняя граница'
                from public.vacancies v
'''

In [17]:
# результат запроса
df = pd.read_sql_query(query_4_3, connection)
df

,Средняя ЗП,Граница
0,71065.0,Нижняя граница
1,110537.0,Верхняя граница


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [18]:
# текст запроса
query_4_4 = f'''select distinct schedule, employment, count(*)
                from public.vacancies
                group by schedule, employment
                order by 3 desc
                limit 5
'''

In [19]:
# результат запроса
df = pd.read_sql_query(query_4_4, connection)
df

,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [20]:
# текст запроса
query_4_5 = f'''select distinct experience, count(*)
                from public.vacancies
                where experience is not null
                group by experience
                order by 2 asc
'''

In [21]:
# результат запроса
df = pd.read_sql_query(query_4_5, connection)
df

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

Ожидаемо количество вакансий больше всего в столицах и крупных городах. Обращает внимание большое число вакансий с возможностью удаленной работы - почти 8000 из 49000 записей. Вероятно, толчок к развитию такого способа работы связан как с опытом карантина, так и с текущей геополитической обстановкой (время формирования датасета неизвеста).

Представленные средние верхняя и нижняя границы ЗП весьма высоки. Для более детального анализа и формирования выводов нужно изучать вакансии с указанными окладами и, вероятно, менять способ подсчета (среднее арифметическое очень сильно искажает действительность).

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [22]:
# текст запроса
query_5_1 = f'''(select distinct e.name, count(*)
                from public.employers e
                join public.vacancies v on e.id = v.employer_id
                group by e.id
                order by 2 desc
                limit 1)
                union
                (select distinct e.name, count(*)
                from public.employers e
                join public.vacancies v on e.id = v.employer_id
                group by e.id
                order by 2 desc
                offset 4
                limit 1)
                order by 2 desc
'''

In [23]:
# результат запроса
df = pd.read_sql_query(query_5_1, connection)
df

,name,count
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [24]:
# текст запроса
query_5_2 = f'''with emp as (
                    select a.name "Регион", count(e.id) "Количество работодателей"
                    from public.areas a
                    left join public.employers e on a.id = e.area
                    group by a.name
                ), vac as (
                    select a.name "Регион", count(v.id) "Количество вакансий"
                    from public.areas a
                    left join public.vacancies v on v.area_id = a.id
                    group by a.name
                )
                select *
                from emp
                natural join vac
                order by 2 desc, 3 desc
                limit 10
'''

In [25]:
# результат запроса
df = pd.read_sql_query(query_5_2, connection)
df

,Регион,Количество работодателей,Количество вакансий
0,Москва,5864,5333
1,Санкт-Петербург,2217,2851
2,Минск,1115,2112
3,Алматы,721,1892
4,Екатеринбург,609,1698
5,Новосибирск,573,2006
6,Казань,480,1415
7,Нижний Новгород,426,1670
8,Россия,410,0
9,Краснодар,409,1301


In [26]:
# ответ на вопрос в модуле
# для этого изменим сортировку и лимит выдачи
query_5_2_1 = f'''with emp as (
                    select a.name "Регион", count(e.id) "Количество работодателей"
                    from public.areas a
                    left join public.employers e on a.id = e.area
                    group by a.name
                ), vac as (
                    select a.name "Регион", count(v.id) "Количество вакансий"
                    from public.areas a
                    left join public.vacancies v on v.area_id = a.id
                    group by a.name
                )
                select *
                from emp
                natural join vac
                order by 3 asc, 2 desc
                limit 1
'''

In [27]:
# результат запроса
df = pd.read_sql_query(query_5_2_1, connection)
df

,Регион,Количество работодателей,Количество вакансий
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [28]:
# текст запроса
query_5_3 = f'''select 
                    e.name "Работодатель", 
                    count(distinct v.area_id) "Количество регионов с вакансиями работодателя"
                from public.vacancies v
                join public.employers e on v.employer_id = e.id
                group by e.id
                order by 2 desc
                limit 10
'''

In [29]:
# результат запроса
df = pd.read_sql_query(query_5_3, connection)
df

,Работодатель,Количество регионов с вакансиями
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
5,Совкомбанк,63
6,МТС,55
7,"ЭФКО, Управляющая компания",49
8,Почта России,48
9,КРОН,48


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [30]:
# текст запроса
query_5_4 = f'''select count(*)
                from public.employers e
                left join public.employers_industries ei on e.id = ei.employer_id
                where ei.industry_id is null
'''

In [31]:
# результат запроса
df = pd.read_sql_query(query_5_4, connection)
df

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [32]:
# текст запроса
# В данном задании фильтрация возможна только после группировки
query_5_5 = f'''select e.name "Название компании"
                from public.employers_industries ei
                join public.employers e on ei.employer_id = e.id
                group by ei.employer_id, e.id
                having count(*) = 4
                order by 1
                offset 2
                limit 1
'''

In [33]:
# результат запроса
df = pd.read_sql_query(query_5_5, connection)
df

,Название компании
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [34]:
# текст запроса
query_5_6 = f'''select count(*)
                from public.employers_industries ei
                join public.industries i on ei.industry_id = i.id
                where i.name = 'Разработка программного обеспечения'
'''

In [35]:
# результат запроса
df = pd.read_sql_query(query_5_6, connection)
df

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [36]:
# код для получения списка городов-миллионников
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8#%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D0%BE%D0%B2-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D0%BE%D0%B2'
# Таблицу после получения преобразуем в кортеж
cities = tuple(pd.read_html(url)[1]['Город'])

In [37]:
# текст запроса
# В условие фильтрации передадим кортеж с городами-миллионниками
query_5_7 = f'''with mill as (
                    select
                        a.name "Город",
                        count(*) "Количество вакансий"
                    from public.vacancies v
                    join public.employers e on v.employer_id = e.id
                    join public.areas a on v.area_id = a.id
                    where e.name = 'Яндекс' and a.name in {cities}
                    group by a.name
                )
                (
                    select *
                    from mill
                )
                union
                (
                    select 'total', sum(mill."Количество вакансий")
                    from mill
                )
                order by 2
'''

In [38]:
# результат запроса
df = pd.read_sql_query(query_5_7, connection)
df

,Город,Количество вакансий
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Уфа,26.0
8,Самара,26.0
9,Краснодар,30.0


***

Наибольшее число вакансий предоставляют наиболее развитые отрасли современной российской экономики. Среди лидеров по количеству вакансий - представители IT, нефтегазового и банковского секторов.

Можно предположить, что крупные компании предоставляют вакансии в разных, в том числе небольших, регионах, однако данное утверждение требует более глубокого анализа. На примере Яндекса - лидера по количеству вакансий - можно увидеть, что в крупнейших городах суммарно всего 485 вакансий из более чем 1900.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [39]:
# текст запроса
query_6_1 = f'''select count(*) "Количество вакансий"
                from public.vacancies v
                where lower(v.name) like '%data%' or lower(v.name) like '%данн%'
'''

In [40]:
# результат запроса
df = pd.read_sql_query(query_6_1, connection)
df

,Количество вакансий
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [41]:
# текст запроса
# В запросе перечислим критерии попадания в выборку (за исключением случайного совпадения с HTML)
query_6_2 = f'''select count(*)
                from public.vacancies v
                where (
                        (
                            lower(v.name) like '%data scientist%' 
                            or lower(v.name) like '%data science%' 
                            or lower(v.name) like '%исследователь данных%' 
                            or v.name like '%ML%'
                            or lower(v.name) like '%machine learning%'
                            or lower(v.name) like '%машинн%обучен%'
                        )
                        and lower(v.name) not like '%html%'
                    ) and (
                        lower(v.name) like '%junior%'
                        or v.experience = 'Нет опыта'
                        or v.employment = 'Стажировка'
                    )
'''

In [42]:
# результат запроса
df = pd.read_sql_query(query_6_2, connection)
df

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [43]:
# текст запроса
query_6_3 = f'''select count(*)
                from public.vacancies v
                where (
                        (
                            lower(v.name) like '%data scientist%' 
                            or lower(v.name) like '%data science%' 
                            or lower(v.name) like '%исследователь данных%' 
                            or v.name like '%ML%'
                            or lower(v.name) like '%machine learning%'
                            or lower(v.name) like '%машинн%обучен%'
                        )
                        and lower(v.name) not like '%html%'
                    ) and (
                        lower(v.key_skills) like '%sql%'
                        or lower(v.key_skills) like '%postgres%'
                    )
'''

In [44]:
# результат запроса
df = pd.read_sql_query(query_6_3, connection)
df

,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [45]:
# текст запроса
query_6_4 = f'''select count(*)
                from public.vacancies v
                where (
                        (
                            lower(v.name) like '%data scientist%' 
                            or lower(v.name) like '%data science%' 
                            or lower(v.name) like '%исследователь данных%' 
                            or v.name like '%ML%'
                            or lower(v.name) like '%machine learning%'
                            or lower(v.name) like '%машинн%обучен%'
                        )
                        and lower(v.name) not like '%html%'
                    ) and (
                        lower(v.key_skills) like '%python%'
                    )
'''

In [46]:
# результат запроса
df = pd.read_sql_query(query_6_4, connection)
df

,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [47]:
# текст запроса
# Для определения количества ключевых навыков, в виду отсутствия прямого инструмента, воспользуемся разностью длинн строк
# до вычитания символа табуляции и после
query_6_5 = f'''select round(avg(length(v.key_skills) - length(replace(v.key_skills, CHR(9), '')) + 1), 2)
                from public.vacancies v
                where (
                        (
                            lower(v.name) like '%data scientist%' 
                            or lower(v.name) like '%data science%' 
                            or lower(v.name) like '%исследователь данных%' 
                            or v.name like '%ML%'
                            or lower(v.name) like '%machine learning%'
                            or lower(v.name) like '%машинн%обучен%'
                        )
                    and lower(v.name) not like '%html%'
                )
'''

In [48]:
# результат запроса
df = pd.read_sql_query(query_6_5, connection)
df

,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [49]:
# текст запроса
# Примечание: в виду выбранного способа (union), для избежания дублирования кусков кода рационально было сформировать новую таблицу, куда вошла фильтрация вакансий
query_6_6 = f'''with ds as (
                    select *
                    from public.vacancies v
                    where (
                            (
                                lower(v.name) like '%data scientist%' 
                                or lower(v.name) like '%data science%' 
                                or lower(v.name) like '%исследователь данных%' 
                                or v.name like '%ML%'
                                or lower(v.name) like '%machine learning%'
                                or lower(v.name) like '%машинн%обучен%'
                            )
                        and lower(v.name) not like '%html%'
                    )
                )
                select experience, round(avg(medium_salary))
                from
                (                    
                    (
                        select experience, (coalesce(salary_from, 0) + coalesce(salary_to, 0)) / 2 as medium_salary
                        from ds
                        where (salary_from is not null) and (salary_to is not null)
                    )                    
                    union all                    
                    (
                        select experience, coalesce(salary_from, 0)
                        from ds
                        where (salary_from is not null) and (salary_to is null)
                    )                    
                    union all                    
                    (
                        select experience, coalesce(salary_to, 0)
                        from ds
                        where (salary_from is null) and (salary_to is not null)
                    )   
                ) as result
                group by experience
                order by 1
'''

In [50]:
# результат запроса
df = pd.read_sql_query(query_6_6, connection)
df

,experience,round
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

Как следует из анализа, менее 1800 вакансий из более чем 49 тысяч имеют отношение к работе с данными. Это почти 4%. Достаточно большая доля, если учесть огромное многообразие различных направлений человеческой деятельности. Именно с DS и ML связано примерно 430 вакансий, 50 из которых - для специалистов без опыта. Если добавить ко всему этому относительно высокую заработную плату, то мы имеем весьма перспективное и открытое для возможностей направление.

# Общий вывод по проекту

Обобщая результаты анализов, можно сказать, что рынок предоставляет достаточно широкие возможности для реализации начинающих специалистов в Data Science в широком смысле этого термина. Можно отдельно отметить весьма быстрый рост заработной платы в зависимости от опыта, а возможности трудоустройства есть не только в мегаполисах, но и в небольших населенных пунктах

# Дополнительные исследования

In [52]:
# Запросим с сервера таблицу, аналогичную предыдущему заданию, но без группировки и "склеенную" с таблицей регионов (areas)
query_add_1 = f'''with ds as (
                    select *
                    from public.vacancies v
                    where (
                            (
                                lower(v.name) like '%data scientist%' 
                                or lower(v.name) like '%data science%' 
                                or lower(v.name) like '%исследователь данных%' 
                                or v.name like '%ML%'
                                or lower(v.name) like '%machine learning%'
                                or lower(v.name) like '%машинн%обучен%'
                            )
                        and lower(v.name) not like '%html%'
                    )
                )
                select *
                from
                (                    
                    (
                        select *, (coalesce(salary_from, 0) + coalesce(salary_to, 0)) / 2 as average_salary
                        from ds
                        where (salary_from is not null) and (salary_to is not null)
                    )                    
                    union all                    
                    (
                        select *, coalesce(salary_from, 0)
                        from ds
                        where (salary_from is not null) and (salary_to is null)
                    )                    
                    union all                    
                    (
                        select *, coalesce(salary_to, 0)
                        from ds
                        where (salary_from is null) and (salary_to is not null)
                    )   
                ) as result
                left join public.areas a on result.area_id = a.id
'''

In [53]:
# результат запроса
df = pd.read_sql_query(query_add_1, connection)
display(df)
df.info()

,id,name,key_skills,schedule,experience,employment,salary_from,salary_to,area_id,employer_id,average_salary,id,name
0,55534594,Middle/Senior ML Specialist,Python\tOpenCV\tPyTorch\tGit\tDocker\tCUDA\tGs...,Полный день,От 1 года до 3 лет,Полная занятость,NaN,175000.0,2,143439,175000,2,Санкт-Петербург
1,55552998,Математик-аналитик (data scientist / data anal...,Python\tМатематическая статистика\tПрогнозиров...,Полный день,Нет опыта,Полная занятость,NaN,98000.0,2,2954482,98000,2,Санкт-Петербург
2,54998664,Machine Learning Engineer,Python\tPyTorch\tTensorflow\tDeep Learning\tCl...,Удаленная работа,От 1 года до 3 лет,Полная занятость,175346.0,NaN,2,4431300,175346,2,Санкт-Петербург
3,54795186,Data Scientist,PyTorch\tPython\tTensorflow\tComputer Vision\t...,Полный день,От 3 до 6 лет,Полная занятость,300000.0,NaN,2,4373268,300000,2,Санкт-Петербург
4,55145641,Junior Data Scientist,Python\tSQL\tMachine Learning\tData Science\tА...,Полный день,Нет опыта,Полная занятость,50000.0,NaN,2,40938,50000,2,Санкт-Петербург
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,55053176,Data Scientist (Machine Learning),Python/R\tData Scientist\tMachine Learning,Полный день,От 3 до 6 лет,Полная занятость,NaN,222104.0,13,851604,222104,13,Армения
58,55034373,Machine Learning Engineer,Python\tAWS\tKubernetes\tAzure\tMachine Learni...,Полный день,От 3 до 6 лет,Полная занятость,NaN,292242.0,13,851604,292242,13,Армения
59,54684143,Data Scientist / Data Analyst в области иннова...,Python\tSQL\tPower BI\tGit\tАналитическое мышл...,Полный день,От 1 года до 3 лет,Полная занятость,NaN,130000.0,2090,40678,130000,2090,Королев
60,55313995,C++ Machine Learning Engineer (Computer Vision),Computer Vision\tC/C++\tMachine learning,Удаленная работа,От 3 до 6 лет,Полная занятость,NaN,233794.0,5046,5970808,233794,5046,Черногория


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              62 non-null     int64  
 1   name            62 non-null     object 
 2   key_skills      58 non-null     object 
 3   schedule        62 non-null     object 
 4   experience      62 non-null     object 
 5   employment      62 non-null     object 
 6   salary_from     46 non-null     float64
 7   salary_to       44 non-null     float64
 8   area_id         62 non-null     int64  
 9   employer_id     62 non-null     int64  
 10  average_salary  62 non-null     int64  
 11  id              62 non-null     int64  
 12  name            62 non-null     object 
dtypes: float64(2), int64(5), object(6)
memory usage: 6.4+ KB


In [54]:
# Выполним преобразования: удалим ненужные признаки, переименуем оставшиеся. Полученная таблица пригодится для дальнейших исследований
median_salary = df.copy()
median_salary = median_salary.drop(['id', 'area_id', 'employer_id'], axis=1)
median_salary.columns = ['Название вакансии', 'Ключевые навыки', 'График', 'Опыт работы', 'Занятость', 'ЗП от, руб.', 'ЗП до, руб.', 'Средняя ЗП, руб.', 'Регион']
display(median_salary)

,Название вакансии,Ключевые навыки,График,Опыт работы,Занятость,"ЗП от, руб.","ЗП до, руб.","Средняя ЗП, руб.",Регион
0,Middle/Senior ML Specialist,Python\tOpenCV\tPyTorch\tGit\tDocker\tCUDA\tGs...,Полный день,От 1 года до 3 лет,Полная занятость,NaN,175000.0,175000,Санкт-Петербург
1,Математик-аналитик (data scientist / data anal...,Python\tМатематическая статистика\tПрогнозиров...,Полный день,Нет опыта,Полная занятость,NaN,98000.0,98000,Санкт-Петербург
2,Machine Learning Engineer,Python\tPyTorch\tTensorflow\tDeep Learning\tCl...,Удаленная работа,От 1 года до 3 лет,Полная занятость,175346.0,NaN,175346,Санкт-Петербург
3,Data Scientist,PyTorch\tPython\tTensorflow\tComputer Vision\t...,Полный день,От 3 до 6 лет,Полная занятость,300000.0,NaN,300000,Санкт-Петербург
4,Junior Data Scientist,Python\tSQL\tMachine Learning\tData Science\tА...,Полный день,Нет опыта,Полная занятость,50000.0,NaN,50000,Санкт-Петербург
...,...,...,...,...,...,...,...,...,...
57,Data Scientist (Machine Learning),Python/R\tData Scientist\tMachine Learning,Полный день,От 3 до 6 лет,Полная занятость,NaN,222104.0,222104,Армения
58,Machine Learning Engineer,Python\tAWS\tKubernetes\tAzure\tMachine Learni...,Полный день,От 3 до 6 лет,Полная занятость,NaN,292242.0,292242,Армения
59,Data Scientist / Data Analyst в области иннова...,Python\tSQL\tPower BI\tGit\tАналитическое мышл...,Полный день,От 1 года до 3 лет,Полная занятость,NaN,130000.0,130000,Королев
60,C++ Machine Learning Engineer (Computer Vision),Computer Vision\tC/C++\tMachine learning,Удаленная работа,От 3 до 6 лет,Полная занятость,NaN,233794.0,233794,Черногория


Проведем исследование отношения ключевых навыков и медианной зарплаты

In [55]:
# Сформируем дополнительную таблицу для анализа ключевых навыков
key_skills = median_salary[['Название вакансии', 'Ключевые навыки', 'Опыт работы', 'Средняя ЗП, руб.']]
# Определим записи с пропущенными значениями в признаке "Ключевые навыки" и удалим их
display((key_skills['Ключевые навыки'].isna()).value_counts())
key_skills.dropna(axis=0, subset=['Ключевые навыки'], inplace=True)
# Разобъем записи о навыках на массив
key_skills['Ключевые навыки'] = key_skills['Ключевые навыки'].apply(lambda x: x.split('\t'))
# Оценим соотношение между количеством ожидаемых навыков и заработной платой
key_skills_numeric = key_skills.copy()
key_skills_numeric['Ключевые навыки'] = key_skills_numeric['Ключевые навыки'].apply(lambda x: len(x))
key_skills_numeric.sort_values(by=['Ключевые навыки', 'Средняя ЗП, руб.'], ascending=False)
key_skills_numeric.groupby(by='Ключевые навыки')['Средняя ЗП, руб.'].median().sort_values(ascending=False)

False    58
True      4
Name: Ключевые навыки, dtype: int64

Ключевые навыки
4     300000.0
14    300000.0
3     229397.0
2     225000.0
16    205000.0
5     200000.0
12    175346.0
8     170000.0
6     150000.0
7     143500.0
10    112500.0
9     108563.0
11     85000.0
Name: Средняя ЗП, руб., dtype: float64

Следует отметить, что в исследовании участвуют только те вакансии, у которых известен размер ЗП. Становится ясно, что количество навыков не связано с предлагаемой ЗП. Это может быть связано с тем, что величина денежного вознаграждения чаще всего определяется собеседованием.

Попробуем оценить, какие же навыки чаще всего ожидаются от соискателей:

In [56]:
# Для этого исследования нам понадобятся все вакансии DS. Запросим их из БД
query_add_2 = f'''select v.key_skills
                from public.vacancies v
                where (
                        (
                            lower(v.name) like '%data scientist%' 
                            or lower(v.name) like '%data science%' 
                            or lower(v.name) like '%исследователь данных%' 
                            or v.name like '%ML%'
                            or lower(v.name) like '%machine learning%'
                            or lower(v.name) like '%машинн%обучен%'
                        )
                    and lower(v.name) not like '%html%'
                )
'''
key_skills_df = pd.read_sql_query(query_add_2, connection)
# Как и ранее, очистим новый датасет от пропусков и разделим строку с навыками на массив
key_skills_df.dropna(axis=0, inplace=True)
key_skills_df['key_skills'] = key_skills_df['key_skills'].apply(lambda x: x.split('\t'))
key_skills_df

,key_skills
0,"[Python, SQL, Работа в команде, OpenCV, ML]"
1,"[Oracle Pl/SQL, Базы данных, Machine Learning,..."
2,"[Python, OpenCV, Tensorflow, Computer Vision, ..."
3,"[Python, Flask, Linux, AWS, Git]"
4,"[Python, SQL, NLP, Scikit-learn, Machine Learn..."
...,...
475,"[Python, Numpy, PyTorch, Pandas, Математически..."
476,"[Python, Scikit-learn, Pandas, Git, Numpy]"
477,"[Qt, ООП, C++, MS Visual Studio, Разработка ко..."
478,"[SCALA, SQL, Математический анализ, Статистиче..."


In [57]:
# Создадим список навыков и заполним его при помощи функции
skills_list = []

def list_filler(list):
    for element in list:
        skills_list.append(element.lower())

key_skills_df['key_skills'].apply(list_filler)

print('20 самых востребованных ключевых навыков: ')
display(Counter(skills_list).most_common(20))
print('Количество уникальных навыков: ', len(set(skills_list)))
print('Всего навыков: ', len(skills_list))

20 самых востребованных ключевых навыков: 


[('python', 348),
 ('sql', 191),
 ('machine learning', 140),
 ('git', 66),
 ('data science', 64),
 ('математическая статистика', 62),
 ('pytorch', 61),
 ('pandas', 55),
 ('data analysis', 54),
 ('ml', 49),
 ('английский язык', 48),
 ('linux', 47),
 ('анализ данных', 46),
 ('tensorflow', 42),
 ('nlp', 42),
 ('big data', 42),
 ('математическое моделирование', 41),
 ('docker', 41),
 ('статистический анализ', 40),
 ('c++', 39)]

Количество уникальных навыков:  389
Всего навыков:  2761


Навык владения Python является самым востребованным и присутствует в 348 из 431 вакансий. Далее SQL и машинное обучение. Также видно наиболее распространенные инструменты и пакеты расширений: Git, PyTorch, TensorFlow, Pandas, Docker.

Следует отметить, что данный анализ годится лишь для примерного представления, каких навыков чаще всего ожидают работодатели: ключевые навыки в вакансиях не приналдежат к фиксированному набору значений, а задаются работодателем. Поэтому, например, "Машинное обучение" может появиться в этом списке под минимум еще двумя названиями: "Machine learning" и "ML". Для частичного исправления результата все записи были приведены к одному регистру, но в действительности для рассмотрения этого вопроса требуется куда более тщательное погружение: например, следует создать большой словарь со списками значений вроде {'Машинное обучение' : ['Machine learning', 'ML']}, с опорой на который преобразовывать навыки именно к фиксированному набору, удобному для анализа.

Кроме того, размывает результат наличие среди навыков умение пользоваться специализированными инструментами, из-за чего такой навык встречается в вакансиях 1-2 раза. Также некоторые навыки являются либо сложносоставными (несколько навыков в одном), либо неинформативными (например, навык "data science" в виду своей чрезвычайно широкой трактовки).

Таким образом, анализ требуемых навыков, заданных работодателями, требует отдельной массивной работы.

Попробуем выявить уровень средней ЗП в зависимости от опыта по двум направлениям: Data Science и ML

In [58]:
query_add_3 = f'''with ds as (
                select *
                from public.vacancies v
                where (
                    lower(v.name) like '%data scientist%' 
                    or lower(v.name) like '%data science%' 
                    or lower(v.name) like '%исследователь данных%'
                    or v.name like '%ML%'
                    or lower(v.name) like '%machine learning%'
                    or lower(v.name) like '%машинн%обучен%'
                )
                and lower(v.name) not like '%html%'
            ), result as (
                (
                    select *, (coalesce(salary_from, 0) + coalesce(salary_to, 0)) / 2 as average_salary
                    from ds
                    where (salary_from is not null) and (salary_to is not null)
                )                    
                union all                    
                (
                    select *, coalesce(salary_from, 0)
                    from ds
                    where (salary_from is not null) and (salary_to is null)
                )                    
                union all                    
                (
                    select *, coalesce(salary_to, 0)
                    from ds
                    where (salary_from is null) and (salary_to is not null)
                )
            )
            select 'Data Science' Направление, result.experience Опыт, round(avg(average_salary)) "Средняя зп, руб."
            from result
            where (
                lower(name) like '%data scientist%' 
                or lower(name) like '%data science%' 
                or lower(name) like '%исследователь данных%'
            )
            group by result.experience
            union all
            select 'Machine Learning', result.experience, round(avg(average_salary))
            from result
            where (
                (
                    name like '%ML%'
                    or lower(name) like '%machine learning%'
                    or lower(name) like '%машинн%обучен%'
                )
                and lower(name) not like '%html%'
            )
            group by result.experience
            order by 1, 2
'''
ds_ml_salary_df = pd.read_sql_query(query_add_3, connection)
ds_ml_salary_df

,Направление,Опыт,"Средняя зп, руб."
0,Data Science,Нет опыта,67917.0
1,Data Science,От 1 года до 3 лет,151156.0
2,Data Science,От 3 до 6 лет,258507.0
3,Machine Learning,Нет опыта,115000.0
4,Machine Learning,От 1 года до 3 лет,133385.0
5,Machine Learning,От 3 до 6 лет,246435.0


Судя по результатам, начинающим специалистам в области машинного обучения готовы платить больше существенно, чем дата сайентистам с тем же опытом. Однако далее ситуация меняется, и дата сайентисту с опытом работы от года до трех в среднем предлагают на примерно 20 тысяч больше. Специалисты с опытом работы от трех до шести лет могут рассчитывать примерно на одинаковую ЗП.

Выборка, над которой проводились исследования, была небольшой: немногим более 60 вакансий, большинство из которых (примерно 40) - для специалистов по DS. Возможно, разрыв в заработных платах для соискателей без опыта связан с именно с малой выборкой.

In [59]:
connection.close()